# Time Series Analysis
#### Notebook Created on: 22 May 2018
##### Last Update: 26 May 2018
##### Version 1.5
##### Author: Vivek Krishnamoorthy

*I have prepared this material with the purpose of introducing **financial econometrics** (the statistical analysis of financial markets) to an audience that has little to no experience in the field.*

*There are a detailed set of references at the end of this document. Time Series Analysis is a fascinating subject and I would encourage you to explore the reference material for a deeper understanding. What we attempt to cover over the next couple of sessions is the proverbial 'tip of the iceberg'.*

### Introduction

Econometrics is the application of statistical methods in economics (and increasingly in other social sciences too). It helps us to evaluate the relationships between different variables. For ex.
[the relationship between average classroom size and scholastic achievement](https://economics.mit.edu/files/8273) (Angrist & Lavy, QJE, 1999)

Least-squares regression is the most widely used method for modeling such relationships. It is known by many other names like "regression", "OLS", or "linear regression". OLS however is primarily used for [cross-sectional data](https://en.wikipedia.org/wiki/Cross-sectional_regression) i.e. data which is associated with a single period or point in time.

### Why Time Series Analysis?

However, **data that we encounter in macroeconomics and empirical finance are often in the form of a time series** i.e. sequential observations of the same variable. Inflation index numbers or stock price observations over time are time series.

In any kind of statistical modelling, we collect a sample of data from the phenomenon or process that we are analyzing. 
* With cross-sectional data, we randomly sample from a population to make statistical inferences. 
* For time series data, we have historical observations of say, the stock price for the last 252 days (if we are modelling the price of a stock at time t). 

It is helpful to think of our sample as the random realization of a ***data generating process***. 


**Time series analysis** can be broadly defined as analyzing the historical data to understand the underlying process, its past behavior and then use it to predict the future behaviour of the process. 

As quantitative traders, we can use these analysis techniques in ***understanding asset price behaviour and make trading decisions based on it***.

We can write a time series as $$\{x_1, x_2, \dots, x_T\}$$ or $$\{x_t\},\: t = 1, 2,\dots, T$$


There is an important concept concerned with time series known as **stationarity**.

## Stationarity

A significant part of the analysis of time series data is about the stationarity of the data. As it turns out, financial time series are mostly non-stationary. So whenever we are analyzing our data, we first need to check for its presence and *'stationarize'* it if it is not so. 

Let us now look at what it means.

A stochastic process $\{x_t\}$ is weakly stationary or covariance stationary if
* $\mathrm{E}(x_t) = \mu \ne f(t)\implies$ The mean of the series is constant and not a function of time.
* $Var(x_t) = \sigma^2 \ne f(t)\implies$ The variance of the series is constant and not a function of time (This is called homoskedasticity).
* $Cov(x_t, x_{t+h}) = g(h) \ne f(t)\implies$ The covariance of the $t^{th}$ term and the $(t+h)^{th}$ depends only on the interval $h$ and is not a function of time.

A stochastic process $\{x_t\}$ is strongly stationary or strictly stationary if the joint probability distribution function of $\{x_{t-s},\dots,x_t,\dots, x_{t+s}\}$ is independent of t for all s. $\implies$ The joint distribution of the random variables is not a function of time.

### Why It Matters

* When a process is stationary, we can make forecasts since its statistical characteristics (such as mean, variance, covariance, etc.) are time invariant.
* In fact, if a time series process is non-stationary, it means that its statistical characteristics vary with time. Therefore any values (such as the mean, variance, etc) that the model would predict would not be reliable.
* In statistical modelling, a number of tests implicitly assume that the data being tested is stationary. So if we run these tests like a 'black-box' on non-stationary data, our results would be useless. 


### Plotting a White Noise process

This is the building block for the basic ARMA models.

Here, we simulate and plot the white noise process $\varepsilon_0$, $\varepsilon_1$,$\dots$, $\varepsilon_T$ where each observation is drawn from a standard normal distribution. All the observations are independent and identically distributed (iid).$$\varepsilon_t \sim N(0, 1)$$

By definition, the errors are serially uncorrelated and the expected mean of these errors is equal to zero.

Any kind of time series models that we build should ideally result in white noise residuals (errors). Why should it be so? Because this would mean that we have successfully fit a model that captures the key dynamics of the underlying process. The only part which is unexplained by the model is random error. In other words, the difference between the values we forecast and the the values that would eventually occur would be residuals that would be indistinguishable from white noise.



` We import all the required libraries below. In case you don't have any of the below, please install them using either the conda or the pip package managers
`

In [1]:
import os
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
from arch import arch_model

import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl

`The below mentioned function plots a time series and helps us analyze important statistical characteristics visually.`

In [2]:
def tsplot(y, lags=None, figsize=(10, 8), style='fivethirtyeight'):
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
    with plt.style.context(style):
        fig = plt.figure(figsize=figsize)
        layout = (3, 2)
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))
        qq_ax = plt.subplot2grid(layout, (2, 0))
        pp_ax = plt.subplot2grid(layout, (2, 1))
        
        y.plot(ax=ts_ax)
        ts_ax.set_title('Time Series Analysis Plots')
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax, alpha=0.05)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax, alpha=0.05)
        sm.qqplot(y, line='s', ax=qq_ax)
        qq_ax.set_title('QQ Plot')
        scs.probplot(y, sparams=(y.mean(), y.std()), plot=pp_ax)
        
        plt.tight_layout()
    
    return


`Let's now model a White Noise process and examine the output`

In [ ]:
np.random.seed(2173)
# Plotting discrete white noise
randser = np.random.normal(size=1000)
tsplot(randser, lags=30)

`From the plots, we can see that the data is distributed randomly and follows a Gaussian white noise process.`

#### Understanding the time series plots

* The first row at the top plots $\epsilon_t$ versus t.
* The second row has the plots of the [autocorrelation function (ACF)](https://en.wikipedia.org/wiki/Autocorrelation) and the [partial autocorrrelation function (PACF)](https://en.wikipedia.org/wiki/Partial_autocorrelation_function). These plots are called correlograms.
    * ACF, as the name suggests, is the correlation of a variable with itself. Here, rather than finding the correlation between two variables, say $x$ and $y$, we compute the correlation between $x_t$ and $x_{t-h}$. We plot these autocorrelations at different lags (1, 2, ...). What interests us is whether there is a pattern in the series or is it truly random. There are two key features that we look for:
        1. Does the ACF plot cut off abruptly?
        2. Does the ACF plot decay gradually/exponentially?
    * PACF, as we would imagine from the name, computes the 'partial' correlation of a variable with itself. Here, we plot the autocorrelations at different lags (1, 2, ...) but after adjusting for the effects of the smaller lags. For ex. the partial autocorrelation calculated at lag 4, would adjust for the effects at lags 1, 2 and 3.

By looking at the correlograms of ACF and PACF, we can detect if any autocorrelation is still present after removing the effects of any deterministic trends, or seasonality effects.

Once we have fitted a time series model, the correlograms help us to judge if a model is a good fit or if need to further refine it to remove any additional serial correlations.

* The third row has [QQ plots](https://www.itl.nist.gov/div898/handbook/eda/section3/qqplot.htm) and the [normal probability plots](https://en.wikipedia.org/wiki/Normal_probability_plot).
    * QQ plots are used to compare the distributions of the errors with a normal distribution. If we find that the dots are more or less on the 45 degree line, then it means that the errors are normally distributed (which is what we'd like to see).
    * Similarly, the normal probability plot also assesses whether or not the data is approximately normally distributed. If the plotted points form an approximately straight line, it indicated normality.


### Why Do We Care So Much about White Noise (It's noise after all, right? :))

Ipso facto, there is nothing particularly interesting about Gaussian White Noise. I'd go so far as to say, it's actually quite boring. If say $\epsilon_t$ is unusually high, there's no reason for $\epsilon_{t+1}$ to continue to stay high or go low. It's literally the equivalent of one coin toss having no bearing on the next. But I digress. The White Noise process does not capture the most interesting feature of time series which is persistence. However, as we'll see, more sophisticated models are created by taking different combinations of $\epsilon_t$.

### Plotting a Random Walk Without a Drift

The random walk time series model can be represented by the equation $$x_t = x_{t-1} + \epsilon_t$$, where $\epsilon_t$ is white noise as defined above.


As we can see from the above plots, the random walk model is a non-stationary process.
If we rearrange the above equation, we get $$\epsilon_t = x_t - x_{t-1}$$

Notice that when we compute the first differences of our series, we get the random walk process!
Calculating first differences is one of the commonest ways of transforming a non-stationary process into a stationary process.

Let's examine the time series analysis plots of the first difference of $x_t$ and see if we get what we expect to see ( a white noise process).


Lo and behold! It's exactly in line with our intuitions. Let's now try the random walk model on some financial data. We will work with the stock returns of Walmart now.

In [30]:
data = pd.read_csv("WMT.csv", index_col="Date", parse_dates=True)
lprice = np.log(data['Close'])
ret = (lprice / lprice.shift(1)).dropna()

` The time series plot looks quite similar to white noise at first glance. Notice the QQ and the probability plots though. Both indicate that the process is approximately normal but with 'fat tails'. Also, when we closely examine the correlograms, there appears to be some significant autocorrelations at lags 1, 2, and 13. This means we could have better models to describe the daily returns process.`

### Basic ARMA Models

These are a class of models which are created by taking linear combinations of white noise.

$$AR(1) : x_t = \phi x_{t-1} + \epsilon_t$$
$$MA(1) : x_t = \epsilon_t + \theta \epsilon_{t-1}$$

$$AR(p) : x_t = \phi_1 x_{t-1} + \phi_2 x_{t-2} + \dots + \phi_p x_{t-p} + \epsilon_t$$
$$MA(q) : x_t = \epsilon_t + \theta_1 \epsilon_{t-1} + \dots + \theta_q \epsilon_q$$

$$ARMA(p, q) : x_t = \phi_1 x_{t-1} + \phi_2 x_{t-2} + \dots + \phi_p x_{t-p} + \epsilon_t + \theta_1 \epsilon_{t-1} + \dots + \theta_q \epsilon_q$$


Notice how all the models that we have seen above will generate sequences $\{x_t\}$ that can be computed with 
* a starting value for the $\{x_t\}$ process, $x_0$
* a sequence of realizations of the white noise process $\{\epsilon_t\}$

Autoregressive models are a multivariate extension of the random walk model. What this means in English is that the current value of the process is a linear combination of past observations.

Moving Average models are very similar to $AR(p)$ models. In $MA(q)$ models, the current value of the process is a linear combination of past error terms.

An even more realistic model combines the characteristics of $AR$ and $MA$ processes and is called the Auto Regressive Moving Average (ARMA) model.

* $AR(p)$ models attempt to explain the mean reversion and trending behaviours that we observe in asset prices.
* $MA(q)$ models try to capture the idiosyncratic shocks that are observed in financial markets. A white noise process models these shocks quite well. We can think of events like terrorist attacks, earnings surprises, sudden political changes, etc. as the random shocks affecting the asset price movements.

You can learn more about these models by going through the relevant references cited at the end of this document.

We will now go through simulations of different processes to get a better feel for them and their key statistical properties.

### Simulation of an $AR(1)$ process with $\phi = 0.75$

### Fitting an $AR(p)$ model

` We use the statsmodels package to estimate the coefficients and also the order. If the AR model is correct, then the estimated phi coefficient should be approximately equal to the true value of 0.75 and the order of the process should be 1.`

In [9]:
mdl = sm.tsa.AR(x).fit(maxlag=30, ic='aic', trend='nc')
est_order = sm.tsa.AR(x).select_order( maxlag=30, ic='aic', trend='nc')

true_order = 1
print('\nalpha estimate: %3.5f | order_estimate %s'%(mdl.params, est_order))
print('\ntrue alpha = %s | true order = %s'%(ϕ, true_order))



alpha estimate: 0.76578 | order_estimate 1

true alpha = 0.75 | true order = 1


### Simulation of an  $MA(1)$  process with $\theta = 0.75$

### Fitting an $MA(q)$ model

` We run a set of steps to estimate the coeeficients of the MA process. Let's try and fit an MA(1) as shown.`

### Simulation and fitting of an $ARMA(2, 2)$ process with phis = [0.5, -0.25] and thetas = [0.5, -0.3]

#### References

1. Cochrane, John H. "Time series for macroeconomics and finance." Manuscript, University of Chicago (2005).
2. http://www.seanabu.com/2016/03/22/time-series-seasonal-ARIMA-model-in-python/
3. Hamilton, James Douglas. Time Series Analysis. Vol. 2. Princeton: Princeton university press, 1994.
4. Tsay, Ruey S. Analysis of financial time series. Vol. 543. John Wiley & Sons, 2005.
5. https://www.quantstart.com/
